In [86]:
import pandas as pd
from langchain_groq import ChatGroq
from decouple import AutoConfig
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [87]:
config = AutoConfig(search_path="../.env")

GROQ_API_KEY = config("GROQ_API_KEY")
LANGCHAIN_TRACING_V2 = config("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = config("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT = config("LANGCHAIN_PROJECT")
lANGCHAIN_API_KEY = config("LANGCHAIN_API_KEY")

In [88]:
web_data = WebBaseLoader(web_path="https://www.amazon.jobs/en/jobs/2853585/senior-software-development-engineer").load()
llm = ChatGroq(model="llama-3.1-70b-versatile", api_key=GROQ_API_KEY, temperature=0.5)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
json_parser = JsonOutputParser()
str_parser = StrOutputParser()

In [89]:
scrape_prompt = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        THe Scraped text is from the career page of a website.
        your job is to extract the job postings and return them in JSON format containing
        the following keys: `role`, `experience`, `skills` and `description`.
        Return only the valid JSON.
        ### VALID JSON (NO PREAMBLE):
    """
)

email_prompt = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    ### INSTRUCTION:
    You are Madhu, a business development executive at Mocha. 
    Mocha is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing 
    the capability of Mocha in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Mocha's portfolio: {link_list}
    Remember you are Madhu, BDE at Mocha. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

In [90]:
scrape_chain = scrape_prompt | llm | json_parser
email_chain = email_prompt | llm | str_parser

In [91]:
print(scrape_prompt.format(**{"page_data":page_data}))


        ### SCRAPED TEXT FROM WEBSITE:
        Senior Software Development Engineer - Job ID: 2853585 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsDiversity, equity, and inclusionInterview tipsLeadership principlesWorking at AmazonFAQ×Senior Software Development EngineerJob ID: 2853585 | ADCI HYD 13 SEZApply nowDESCRIPTIONWould you like to work on one of the world's largest transactional distributed systems? How about working with customers and peers from the entire range of Amazon's business on cool new features? Whether you're passionate about building highly scalable and reliable systems or a software developer who likes to solve business problems, Amazon Tax Platform Services is the place for you.We are responsible for the tax platform, providing the core services that calculate taxes (sales tax and VAT) for all Amazon sales, physical and digital, global

In [92]:
page_data = web_data.pop().page_content
response = scrape_chain.invoke(input={"page_data": page_data})
print(response)

{'role': 'Senior Software Development Engineer', 'experience': '5+ years', 'skills': ['Java', 'Linux', 'Oracle', 'programming with at least one software programming language', 'leading design or architecture (design patterns, reliability and scaling) of new and existing systems', 'full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations'], 'description': 'We are looking for software engineers who thrive on complex problems and relish the challenge of operating complex and mission critical systems under extreme loads. Our systems manage hundreds of millions of records, and respond to millions of service requests per minute.'}


In [93]:
df = pd.read_csv("../resources/my_portfolio.csv")
display(df)

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [94]:
csv_documents = []
for _, row in df.iterrows():
    csv_documents.append(Document(page_content=row["Techstack"], metadata={"source": row["Links"]}))

In [95]:
vector_store = Chroma(collection_name="portfolio_collection", embedding_function=embeddings)
vector_store.add_documents(csv_documents)

['595cb317-f9b1-4694-9bdc-8571b20e584d',
 '918c9418-4c82-4b71-8f24-806f4effff15',
 'dddb05fe-e47f-4458-9242-a9ef358116ad',
 '2a31e305-09ae-4091-ba1d-40416ecbdbfe',
 '3a64fb8a-45a5-4c89-ba4d-82e5a725cd86',
 'ba0b6aac-dbda-4db3-ae14-18acdb7ba2b4',
 '02b82fce-6f55-4596-93b5-2e8e36ea90a8',
 'b41f6739-31b7-44df-91ea-6ceb96f7f964',
 '94539586-5367-47f1-a630-38ef6f2f31dc',
 '94ff151b-6fe8-4020-9b80-009d9338741e',
 '7059eaac-12a2-465e-988c-c16d2b814c21',
 '8ae9ec0e-f7c6-42f1-bf53-be0832d90f06',
 '2afe3958-ca03-4cd9-a289-f921a6a26390',
 'f8c1fadd-abbd-4ba5-ae2b-795f1405dd74',
 '50a8c721-6ef7-4412-970f-2977c7c858d1',
 '9fd05de8-eebb-4e09-85b8-73bb12c0798a',
 'a2088b41-2110-44b8-ae69-d85c550834f7',
 '43a77892-f732-4bef-84e9-b500d728e289',
 'f5056c76-56bd-40c3-ae27-307c83ab7696',
 '75052c43-4fb8-4cfa-a860-94778b8eae4c']

In [96]:
relevant_portfolios = []
for skill in response.get('skills'):
    relevant_portfolios.append(vector_store.similarity_search(skill, k=1)[0].metadata['source'])
relevant_portfolios

['https://example.com/java-portfolio',
 'https://example.com/java-portfolio',
 'https://example.com/java-portfolio',
 'https://example.com/ml-python-portfolio',
 'https://example.com/android-portfolio',
 'https://example.com/magento-portfolio']

In [97]:
print(email_prompt.format(**{"job_description":str(response), "link_list":set(relevant_portfolios)}))


    ### JOB DESCRIPTION:
    {'role': 'Senior Software Development Engineer', 'experience': '5+ years', 'skills': ['Java', 'Linux', 'Oracle', 'programming with at least one software programming language', 'leading design or architecture (design patterns, reliability and scaling) of new and existing systems', 'full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations'], 'description': 'We are looking for software engineers who thrive on complex problems and relish the challenge of operating complex and mission critical systems under extreme loads. Our systems manage hundreds of millions of records, and respond to millions of service requests per minute.'}
    ### INSTRUCTION:
    You are Madhu, a business development executive at Mocha. 
    Mocha is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over o

In [98]:
response = email_chain.invoke({"job_description":str(response), "link_list":set(relevant_portfolios)})
print(response)

Subject: Expert Software Development Solutions for Complex Systems

Dear Hiring Manager,

I came across your job posting for a Senior Software Development Engineer and was impressed by the complexity and scale of the systems you're managing. At Mocha, we specialize in empowering enterprises like yours with tailored software solutions that drive scalability, process optimization, cost reduction, and heightened overall efficiency.

Our team of expert software engineers has extensive experience in designing and developing complex systems, leveraging technologies like Java, Linux, and Oracle. We've successfully implemented full software development life cycles, including coding standards, code reviews, source control management, build processes, testing, and operations.

Our portfolio showcases our capabilities in Java-based solutions, which I believe would be of particular interest to you. You can explore our Java portfolio at https://example.com/java-portfolio. Additionally, I'd like to 